In [18]:
import pandas as pd

In [30]:
df= pd.read_csv("r2_df.csv")


In [20]:
value=0
for r2 in df["R2"]:
    value = value + r2
print(value/44)

0.9630302480365702


In [27]:
df=19


## implementation of paper 
title: 



In [31]:
import pandas as pd
import numpy as np

# 1. Look-up Table 1: Regression coefficients [b, a]
# Format: {month: [[b, a], [b, a], [b, a]]}
# Altitude classes:
# 0: >= 2000 m
# 1: 1400–2000 m
# 2: < 1400 m

table_1 = {
    11: [[np.nan, np.nan], [7,  np.nan], [8,  np.nan]],   # Nov
    12: [[np.nan, np.nan], [np.nan, np.nan], [9,  np.nan]],  # Dec
    1:  [[np.nan, np.nan], [np.nan, np.nan], [10, np.nan]],  # Jan
    2:  [[np.nan, np.nan], [np.nan, np.nan], [11, np.nan]],  # Feb
    3:  [[9,  np.nan], [10, np.nan], [12, np.nan]],       # Mar
    4:  [[11, np.nan], [13, np.nan], [14, np.nan]],       # Apr
    5:  [[6,  np.nan], [15, np.nan], [4,  np.nan]],       # May
    6:  [[16, np.nan], [np.nan, np.nan], [np.nan, np.nan]],  # Jun
    7:  [[13, np.nan], [np.nan, np.nan], [np.nan, np.nan]]   # Jul
}

# 2. Look-up Table 2: Region-specific offsets (kg/m3)
region_offsets = {
    1: 7.6,
    2: 11.7,
    3: 11.8,
    4: -1.1,
    5: -0.3,
    6: 12.1,
    7: -14.7
}


def estimate_swe_jonas(df):
    """
    Implements the Jonas et al. (2009) methodology.
    Requires columns:
    - 'date'
    - 'HS_[m]'
    - 'altitude'
    - 'region'
    """

    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month

    def calculate_row_swe(row):
        hs = row['HS_[m]']
        month = row['month']
        altitude = row['altitude']
        region = row['region']

        # No snow or month not available
        if hs <= 0 or month not in table_1:
            return 0.0

        # Determine altitude class
        if altitude >= 2000:
            alt_class = 0
        elif altitude >= 1400:
            alt_class = 1
        else:
            alt_class = 2

        b, a = table_1[month][alt_class]

        # Handle missing coefficients
        if np.isnan(b) or np.isnan(a):
            return np.nan

        # Step 1: Preliminary bulk density
        qb_prel = a * hs + b

        # Step 2: Apply regional offset


        offset = region_offsets.get(region, 0)
        qb_mod = qb_prel + offset

        # Step 3: SWE calculation (kg/m² == mm)
        swe_kg_m2 = hs * qb_mod



        # Convert mm to meters
        return swe_kg_m2 / 1000.0

    df['SWE_estimated_[m]'] = df.apply(calculate_row_swe, axis=1)
    return df


In [ ]:
df_elv=pd.read_csv("coordinates_swe2hs_paper_stations.csv")
df=pd.read_csv("C")
df['elevation'] = 0
df["site_id"].nunique()
df_elv

,site_id,location_name,lon_[wgs84],lat_[wgs84],elevation_[m],dataset
0,1AD,Adelboden,7.560705,46.492044,1325,calibration_data_manual_stations
1,1GA,Gadmen,8.352310,46.736667,1190,calibration_data_manual_stations
2,1GB,Grindelwald Bort,8.051395,46.647282,1565,calibration_data_manual_stations
3,1GD,Grindel,8.064320,46.657918,1950,calibration_data_manual_stations
4,1GS,Gsteig,7.278514,46.376429,1195,calibration_data_manual_stations
...,...,...,...,...,...,...
109,WAL_aws,wattener_lizum,11.638580,47.168200,1995,validation_data_automatic_stations
110,FEL_aws,fellhorn,10.220330,47.339910,1610,validation_data_automatic_stations
111,KUR_aws,kuehroint,12.959730,47.570140,1420,validation_data_automatic_stations
112,SPI_aws,spitzingsee,11.886170,47.671550,1100,validation_data_automatic_stations


In [64]:

for i in range(len(df)):
    site_id = df.loc[i, 'site_id']
    if site_id in df_elv['site_id'].values:
        df.loc[i, 'elevation'] = df_elv.loc[
            df_elv['site_id'] == site_id, 'elevation_[m]'
        ].values[0]
        


,date,HS_[m],SWE_[m],SWE_from_profile_[m],site_id,elevation
0,1991-10-18,0.00,0.000000,NaN,1AD,1325
1,1991-10-19,0.13,0.009007,NaN,1AD,1325
2,1991-10-20,0.06,0.009007,NaN,1AD,1325
3,1991-10-21,0.02,0.007656,NaN,1AD,1325
4,1991-10-22,0.00,0.000000,NaN,1AD,1325
...,...,...,...,...,...,...
223545,2020-04-02,0.07,0.023492,NaN,7ZU,1710
223546,2020-04-03,0.00,0.000000,NaN,7ZU,1710
223547,2020-04-30,0.00,0.000000,NaN,7ZU,1710
223548,2020-05-01,0.01,0.000798,NaN,7ZU,1710


In [ ]:
df.to_csv("coordinates_swe2hs_paper_stations_with_alltitude.csv")
df

,date,HS_[m],SWE_[m],SWE_from_profile_[m],site_id,elevation
0,1991-10-18,0.00,0.000000,NaN,1AD,1325
1,1991-10-19,0.13,0.009007,NaN,1AD,1325
2,1991-10-20,0.06,0.009007,NaN,1AD,1325
3,1991-10-21,0.02,0.007656,NaN,1AD,1325
4,1991-10-22,0.00,0.000000,NaN,1AD,1325
...,...,...,...,...,...,...
223545,2020-04-02,0.07,0.023492,NaN,7ZU,1710
223546,2020-04-03,0.00,0.000000,NaN,7ZU,1710
223547,2020-04-30,0.00,0.000000,NaN,7ZU,1710
223548,2020-05-01,0.01,0.000798,NaN,7ZU,1710


In [74]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --------------------------------------------------
# LOAD DATA
# --------------------------------------------------
df = pd.read_csv("coordinates_swe2hs_paper_stations_with_alltitude.csv")

# --------------------------------------------------
# JONAS SWE COEFFICIENT TABLE
# Format: [b, a]
# --------------------------------------------------
table_1 = {
    11: [[np.nan, np.nan], [7,  np.nan], [8,  np.nan]],
    12: [[np.nan, np.nan], [np.nan, np.nan], [9,  np.nan]],
    1:  [[np.nan, np.nan], [np.nan, np.nan], [10, np.nan]],
    2:  [[np.nan, np.nan], [np.nan, np.nan], [11, np.nan]],
    3:  [[9,  np.nan], [10, np.nan], [12, np.nan]],
    4:  [[11, np.nan], [13, np.nan], [14, np.nan]],
    5:  [[6,  np.nan], [15, np.nan], [4,  np.nan]],
    6:  [[16, np.nan], [np.nan, np.nan], [np.nan, np.nan]],
    7:  [[13, np.nan], [np.nan, np.nan], [np.nan, np.nan]]
}

# --------------------------------------------------
# JONAS SWE MODEL (NaN-SAFE)
# --------------------------------------------------
def estimate_swe_jonas(df):

    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month

    def calculate_row_swe(row):
        hs = row['HS_[m]']
        month = row['month']
        altitude = row['elevation']

        # Invalid snow or month
        if pd.isna(hs) or hs <= 0 or month not in table_1:
            return np.nan

        # Altitude class
        if altitude >= 2000:
            alt_class = 0
        elif altitude >= 1400:
            alt_class = 1
        else:
            alt_class = 2

        b, a = table_1[month][alt_class]

        # If no coefficient exists
        if np.isnan(b):
            return np.nan

        # Jonas formulation (allow b-only)
        if np.isnan(a):
            qb = b
        else:
            qb = a * hs + b

        # SWE (kg/m² == mm)
        return (hs * qb) / 1000.0

    df['SWE_estimated_[m]'] = df.apply(calculate_row_swe, axis=1)
    return df


# --------------------------------------------------
# APPLY MODEL
# --------------------------------------------------
df = estimate_swe_jonas(df)

# --------------------------------------------------
# DIAGNOSTICS
# --------------------------------------------------
print("Total rows:", len(df))
print("Valid SWE estimates:", df['SWE_estimated_[m]'].notna().sum())
print("Valid observed SWE:", df['SWE_[m]'].notna().sum())

# --------------------------------------------------
# ACCURACY METRICS
# --------------------------------------------------
df_eval = df.dropna(subset=['SWE_[m]', 'SWE_estimated_[m]'])

if len(df_eval) == 0:
    print("\n❌ No valid data points for evaluation.")
else:
    obs = df_eval['SWE_[m]'].values
    est = df_eval['SWE_estimated_[m]'].values

    mae = mean_absolute_error(obs, est)
    rmse = np.sqrt(mean_squared_error(obs, est))
    r2 = r2_score(obs, est)
    bias = np.mean(est - obs)

    print("\n✅ Jonas SWE model accuracy (no region):")
    print(f"MAE  : {mae:.4f} m")
    print(f"RMSE : {rmse:.4f} m")
    print(f"R²   : {r2:.3f}")
    print(f"Bias : {bias:.4f} m")



Total rows: 223550
Valid SWE estimates: 122837
Valid observed SWE: 223550

✅ Jonas SWE model accuracy (no region):
MAE  : 0.1937 m
RMSE : 0.2606 m
R²   : -1.096
Bias : -0.1937 m
